# Detect Face

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决无法显示符号的问题
sns.set(font='SimHei', font_scale=1.2, style="ticks")  # 解决Seaborn中文显示问题

def read_infrared_data(file):
  lines = open(file, "r", encoding='utf-8')
  data = []
  for line in lines:
    vs = line.strip().split(",")
    row = [float(v) for v in vs]
    data.append(row)
  return data


def plot_results(part,list_node,width,height,save_root=None):
    list_x=[]
    list_y=[]
    list_temp=[]
    for node in list_node:
        if part=='face':
            for k in node:
                if type(node[k])==tuple and k!="box":
                    x=node[k][0]
                    y=node[k][1]
                    list_x.append(x)
                    list_y.append(y)
                    list_temp.append(node[k][2])
        elif part=="shape":
            # if node['temp']<self.sb_min_temp.value():
            #    continue
            list_x.append(node['x'])
            list_y.append(node['y'])
            list_temp.append(node['temp'])
        else:
            if node['temp']==-1:
                continue
            list_x.append(node['x'])
            list_y.append(node['y'])
            list_temp.append(node['temp'])
    # plt.title('')
    fig = plt.figure(dpi=100)
    axes = fig.add_subplot(111)
    axes.set_xlim(1, width)
    axes.set_ylim(1, height)
    axes.invert_yaxis()
    axes.axis("off")

    axes.scatter(x=list_x, y=list_y,s=list_temp,alpha=0.5,c=list_temp,cmap='inferno')
    if save_root!=None:
        plt.savefig(f"{save_root}/landmarks_{part}.jpg",dpi=300,bbox_inches='tight', pad_inches=0)
    # plt.show()

In [2]:
import os
from IPython.display import Image, display
from matplotlib.figure import Figure

from tiai.mediapip.face import detect_face

root_path=r'datasets-thermal-face'

jpg_path=f'{root_path}/thermal_sq/irface_sub001_seq02_frm00104.jpg_lfb.png'
csv_path=f'{root_path}/csv/irface_sub001_seq02_frm00104.jpg_lfb.csv'
save_detected_path='mediapipe_results/test1/tes1.jpg'
# jpg_path="datasets/测试/test-1.jpg"
img,list_node,list_connection=detect_face(image_path=jpg_path,save_detect_path=save_detected_path,show=True,csv_path=csv_path,min_detection_confidence=0.6,model_selection=0)
print(len(list_node),len(list_connection))
print()
print("Results:")
if len(list_node)!=0:
    face_model=list_node[0]
    print("score: ", face_model['score'][0])
    for part in ['nose_tip','left_eye','right_eye','left_ear_tragion','right_ear_tragion','mouth_center']:
        temp=face_model[part][2]
        print(part,temp)
    print()
    for connection in list_connection:
        print(connection)
# plot
img_matrix = read_infrared_data(csv_path)
img_m_width = len(img_matrix[0])
img_m_height = len(img_matrix)
# plot_results("face",list_node,width=img_m_width,height=img_m_height,save_root='mediapipe_results/test1')




matrix:  768 768
shape:  768 768
box:  xmin: 0.3843526840209961
ymin: 0.25769826769828796
width: 0.33414632081985474
height: 0.334127813577652

noise tip:  0.5404043197631836 0.37342917919158936
score:  [0.841826319694519]
cx=414,cy=285
cx=474,cy=244
cx=355,cy=251
cx=546,cy=293
cx=300,cy=308
cx=418,cy=358
1 0

Results:
score:  0.841826319694519
nose_tip 0.698
left_eye 0.796
right_eye 0.831
left_ear_tragion 0.251
right_ear_tragion 0.604
mouth_center 0.78



In [3]:
def get_face_scores(jpg_path,csv_path,show=False):
    img,list_node,list_connection=detect_face(image_path=jpg_path,save_detect_path=None,show=show,csv_path=csv_path,min_detection_confidence=0.6,model_selection=0)
    score_model={}
    score=-1
    if len(list_node)!=0:
        face_model=list_node[0]
        # print("score: ", face_model['score'][0])
        score=face_model['score'][0]
        for part in ['nose_tip','left_eye','right_eye','left_ear_tragion','right_ear_tragion','mouth_center']:
            temp=face_model[part][2]
            # print(part,temp)
            score_model[part]=temp
    return score, score_model

In [4]:
# batch evaluation
import os
from quickcsv import *
from tqdm import tqdm



root_path=r'datasets-thermal-face'

list_scores=[]
n_failure=0
for file in os.listdir(f'{root_path}/thermal'):
    filename,ext=os.path.splitext(file)
    jpg_path=os.path.join(root_path,'thermal',file)
    csv_path=os.path.join(root_path,'csv',file.replace('.png','.csv'))
    if os.path.exists(jpg_path) and os.path.exists(csv_path):
        score,scores=get_face_scores(jpg_path,csv_path)
        if score!=-1 and scores!=None:
            scores['overall']=score
            scores['filename']=filename
            list_scores.append(scores)
        else:
            n_failure+=1
        print(score,scores)
        print()
percent=len(list_scores)/len(tqdm(os.listdir(f'{root_path}/thermal')))
write_csv('mediapipe_results/test1/test1.csv',list_scores)

print("Success rate: ",percent)




matrix:  768 768
shape:  1024 768
Not found!
-1 {}

matrix:  768 768
shape:  1024 768
Not found!
-1 {}

matrix:  768 768
shape:  1024 768
box:  xmin: 0.4108526408672333
ymin: 0.2482416182756424
width: 0.24971714615821838
height: 0.33295631408691406

noise tip:  0.5294147729873657 0.3656798005104065
score:  [0.7833022475242615]
cx=405,cy=279
cx=452,cy=246
cx=364,cy=240
cx=504,cy=301
cx=317,cy=292
cx=404,cy=350
0.7833022475242615 {'nose_tip': 0.71, 'left_eye': 0.863, 'right_eye': 0.875, 'left_ear_tragion': 0.753, 'right_ear_tragion': 0.741, 'mouth_center': 0.773, 'overall': 0.7833022475242615, 'filename': 'irface_sub001_seq02_frm00104.jpg_lfb'}

matrix:  768 768
shape:  1024 768
box:  xmin: 0.4253533184528351
ymin: 0.25031912326812744
width: 0.2068781554698944
height: 0.2758375406265259

noise tip:  0.4511928856372833 0.36672091484069824
score:  [0.6691847443580627]
cx=345,cy=280
cx=395,cy=236
cx=341,cy=245
cx=483,cy=263
cx=353,cy=279
cx=360,cy=334
0.6691847443580627 {'nose_tip': 0.62, '

  0%|          | 0/2935 [00:00<?, ?it/s]

Write CSV: ['nose_tip', 'left_eye', 'right_eye', 'left_ear_tragion', 'right_ear_tragion', 'mouth_center', 'overall', 'filename']  -> (mediapipe_results/test1/test1.csv)
Success rate:  0.06269165247018739


In [5]:
# single evaluation for FLIR images

root_path='datasets-processed/FLIR_with_same_size'

image_id='FLIR_20220323_014429'

optical_path=os.path.join(root_path,'optical',image_id+'.jpg')

thermal_path=os.path.join(root_path,'thermal_sq',image_id+".jpg")

csv_path=os.path.join(root_path,'csv',image_id+".csv")

score,scores=get_face_scores(optical_path,csv_path,show=True)

print(score,scores)

score1,scores1=get_face_scores(thermal_path,csv_path,show=True)



matrix:  480 640
shape:  1080 1440
box:  xmin: 0.32438284158706665
ymin: 0.29754215478897095
width: 0.3245391249656677
height: 0.24340397119522095

noise tip:  0.4397331774234772 0.4285987615585327
score:  [0.9351156949996948]
cx=210,cy=273
cx=242,cy=226
cx=182,cy=244
cx=302,cy=226
cx=171,cy=261
cx=223,cy=303


0.9351156949996948 {'nose_tip': 34.90229967950745, 'left_eye': 35.33311861000095, 'right_eye': 34.654618872403205, 'left_ear_tragion': 35.175693515600756, 'right_ear_tragion': 34.01974141349484, 'mouth_center': 35.20194634284434}
matrix:  480 640
shape:  1080 1080
box:  xmin: 0.16787046194076538
ymin: 0.1624247133731842
width: 0.5166218876838684
height: 0.516533374786377

noise tip:  0.29859381914138794 0.43988609313964844
score:  [0.6495054960250854]
cx=142,cy=280
cx=211,cy=186
cx=118,cy=210
cx=317,cy=199
cx=118,cy=241
cx=164,cy=350


In [6]:
# batch evaluation

root_path='datasets-processed/FLIR_with_same_size'

list_evals1=[]
list_evals2=[]
list_evals=[]
for file in os.listdir(root_path+"/optical"):
    image_id,ext=os.path.splitext(file)

    optical_path=os.path.join(root_path,'optical',image_id+'.jpg')

    thermal_path=os.path.join(root_path,'thermal_sq',image_id+".jpg")

    csv_path=os.path.join(root_path,'csv',image_id+".csv")

    score1,scores1=get_face_scores(optical_path,csv_path,show=False)
    if score1==-1:
        scores1={}
        for l in "nose_tip,left_eye,right_eye,left_ear_tragion,right_ear_tragion,mouth_center".split(','):
            scores1[l]=0
    
    print(score,scores)

    score2,scores2=get_face_scores(thermal_path,csv_path,show=False)
    
    if score2==-1:
        score2=0
        scores2={}
        for l in "nose_tip,left_eye,right_eye,left_ear_tragion,right_ear_tragion,mouth_center".split(','):
            scores2[l]=0
    
    if score1==-1:
        score1=0
    
    scores1['overall']=score1
    scores1['image_id']=image_id
    
    scores2['overall']=score2
    scores2['image_id']=image_id
    
    list_evals1.append(scores1)
    list_evals2.append(scores2)
    
    score_diff=score1-score2
    
    # for l in "nose_tip,left_eye,right_e ye,left_ear_tragion,right_ear_tragion,mouth_center".split(','):
    #     scores_diff[l]=scores1[l]-scores2[l]
    list_evals.append({'overall_diff':score_diff})
    
write_csv('mediapipe_results/test1/test_flir_optical.csv',list_evals1)
write_csv('mediapipe_results/test1/test_flir_thermal.csv',list_evals2)
write_csv('mediapipe_results/test1/test_flir_diff.csv',list_evals)


    



matrix:  480 640
shape:  1080 1440
box:  xmin: 0.39120569825172424
ymin: 0.43152084946632385
width: 0.2575746476650238
height: 0.19318071007728577

noise tip:  0.5694584846496582 0.5473415851593018
score:  [0.8498564958572388]
cx=272,cy=349
cx=292,cy=325
cx=243,cy=309
cx=293,cy=332
cx=186,cy=299
cx=257,cy=370
0.9351156949996948 {'nose_tip': 34.90229967950745, 'left_eye': 35.33311861000095, 'right_eye': 34.654618872403205, 'left_ear_tragion': 35.175693515600756, 'right_ear_tragion': 34.01974141349484, 'mouth_center': 35.20194634284434}
matrix:  480 640
shape:  1080 1080
Not found!
matrix:  480 640
shape:  1080 1440
box:  xmin: 0.3307499587535858
ymin: 0.32721325755119324
width: 0.21054533123970032
height: 0.15790876746177673

noise tip:  0.46422842144966125 0.41134804487228394
score:  [0.6230447888374329]
cx=221,cy=262
cx=200,cy=239
cx=189,cy=263
cx=198,cy=227
cx=170,cy=287
cx=236,cy=272
0.9351156949996948 {'nose_tip': 34.90229967950745, 'left_eye': 35.33311861000095, 'right_eye': 34.65

In [14]:
# create different images for emissivity
import flyr

root_path='datasets-processed/FLIR_with_same_size'

list_model=[]
for file in os.listdir(os.path.join(root_path,'raw'))[:10]:
    image_id,ext =os.path.splitext(file)
    raw_path=os.path.join(root_path,'raw',file) 
    csv_path=os.path.join(root_path,'csv',image_id+".csv")
    emissivities = [0.6, 0.7, 0.8, 0.9, 1.0]
    # renders=[]
    for e in emissivities:
        thermogram = flyr.unpack(raw_path)
        thermogram = thermogram.adjust_metadata(emissivity=e)
        # thermal = thermogram.celsius  # Access updated data as normal
        render = thermogram.render_pil(
            min_v=27.1,
            max_v=35.6,
            unit="celsius",
            palette="inferno",
        )
        # renders.append(render)
        temp_path=os.path.join(root_path,'temp','render1.jpg')
        render.save(temp_path)
        
        score2,scores2=get_face_scores(temp_path,csv_path,show=False)
        
        print(score2,scores2)
        print()
        if score2==-1:
            score2=0
        list_model.append({
            "image_id":image_id,
            "emissivity":e,
            "score":score2
        })

    # show
    '''
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for render, ax in zip(renders, axes.ravel()):
        ax.grid(False)
        ax.axis('off')
        ax.imshow(render)
    '''

write_csv('mediapipe_results/test1/test_emissivity.csv',list_model)

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
box:  xmin: 0.12782590091228485
ymin: 0.24002617597579956
width: 0.6507667303085327
height: 0.48799341917037964

noise tip:  0.3331909775733948 0.5020896196365356
score:  [0.609440803527832]
cx=158,cy=320
cx=271,cy=269
cx=144,cy=226
cx=357,cy=325
cx=101,cy=233
cx=144,cy=384
0.609440803527832 {'nose_tip': 33.52520924975079, 'left_eye': 31.773990688089384, 'right_eye': 28.43617241156079, 'left_ear_tragion': 26.30348710308823, 'right_ear_tragion': 26.411726583599886, 'mouth_center': 30.42821644738291}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

mat

In [12]:
# create different images for percentile
import flyr

root_path='datasets-processed/FLIR_with_same_size'

list_model=[]
for file in os.listdir(os.path.join(root_path,'raw'))[:10]:
    image_id,ext =os.path.splitext(file)
    raw_path=os.path.join(root_path,'raw',file) 
    csv_path=os.path.join(root_path,'csv',image_id+".csv")

    # renders=[]
    for i in range(1,5+1):
    
        thermogram = flyr.unpack(raw_path)
        thermogram = thermogram.adjust_metadata(emissivity=e)
        # thermal = thermogram.celsius  # Access updated data as normal
        render=thermogram.render_pil(
            min_v=0.0,
            max_v=0.2 * i,
            unit="percentiles",
            palette="inferno",
        )
        # renders.append(render)
        temp_path=os.path.join(root_path,'temp','render2.jpg')
        render.save(temp_path)
        
        score2,scores2=get_face_scores(temp_path,csv_path,show=False)
        
        print(score2,scores2)
        print()
        if score2==-1:
            score2=0
        list_model.append({
            "image_id":image_id,
            "emissivity":e,
            "score":score2
        })

    # show
    '''
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for render, ax in zip(renders, axes.ravel()):
        ax.grid(False)
        ax.axis('off')
        ax.imshow(render)
    '''

write_csv('mediapipe_results/test1/test_percentiles.csv',list_model)

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
box:  xmin: 0.06437428295612335
ymin: 0.1258862167596817
width: 0.9937766194343567
height: 0.7452909350395203

noise tip:  0.7581730484962463 0.4573957026004791
score:  [0.6321431398391724]
cx=362,cy=291
cx=426,cy=235
cx=245,cy=178
cx=417,cy=322
cx=48,cy=220
cx=311,cy=406
0.6321431398391724 {'nose_tip': 26.046681109783606, 'left_eye': 26.09238182626075, 'right_eye': 26.20084067807477, 'left_ear_tragion': 26.023823221101054, 'right_ear_tragion': 26.29778717522214, 'mouth_center': 30.362407078438594}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

matrix:  480 640
shape:  480 640
Not found!
-1 {}

mat